### pymupdf4llm + llama_reader

In [ ]:
from langchain_core.documents import Document
import os
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
import pymupdf4llm

file_path = './Data/2025년 AI반도체 조기 상용화 및 AX실증 지원 사업 통합 공고문.pdf'

llama_reader = pymupdf4llm.LlamaMarkdownReader()
llama_docs = llama_reader.load_data(file_path)

# Document 객체로 변환
documents = []
for doc in llama_docs:
    documents.append(Document(
        page_content=doc.text,
        metadata={
            "source": file_path,
            "page_number": doc.metadata.get("page_number", None)  # 필요한 경우 페이지 번호 추가
        }
    ))

# 예시 출력
print(documents[1].page_content)


### partition_pdf

In [ ]:
import os
from unstructured.partition.pdf import partition_pdf
from PIL import Image
import pytesseract
from langchain_core.documents import Document
from collections import defaultdict

def group_elements_by_page(elements):
    """
    요소들을 페이지 번호 기준으로 그룹화
    """
    page_elements = defaultdict(list)
    for el in elements:
        page_number = getattr(el.metadata, "page_number", 0)
        page_elements[page_number].append(el)
    return page_elements


def convert_to_documents_by_page(elements, source):
    """
    페이지별로 텍스트와 표를 통합하여 Document 객체 생성
    """
    grouped = group_elements_by_page(elements)
    documents = []

    for page_number, elements in grouped.items():
        combined_content = ""
        for el in elements:
            el_type = type(el).__name__
            if el_type in ["CompositeElement", "Table"]:
                combined_content += str(el).strip() + "\n\n"
        documents.append(Document(
            page_content=combined_content.strip(),
            metadata={
                "source": source,
                "page_number": page_number
            }
        ))

    return documents


def extract_pdf_elements(path, fname):
    """
    PDF에서 OCR을 포함해 텍스트와 표를 추출합니다.
    이미지 저장 경로는 path에 저장됩니다.
    """
    elements = partition_pdf(
        filename=os.path.join(path, fname),
        extract_images_in_pdf=False,   # 이미지 저장은 하지 않음
        infer_table_structure=True,    # 표 구조 인식
        chunking_strategy="by_title",  # 제목 기준 나누기
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
        ocr_strategy="auto",           # OCR 자동 적용
        pdf_infer_table_structure=True,
        ocr_languages="kor"
    )
    return elements


def categorize_elements(raw_pdf_elements):
    """
    요소를 테이블과 텍스트로 분리하여 반환합니다.
    """
    tables = []
    texts = []

    for element in raw_pdf_elements:
        type_name = type(element).__name__
        if type_name == "Table":
            tables.append(str(element))
        elif type_name == "CompositeElement":
            texts.append(str(element))

    return texts, tables

# 경로와 파일명
pdf_path = "./data"
pdf_name = "2025년 AI반도체 조기 상용화 및 AX실증 지원 사업 통합 공고문.pdf"

# PDF 요소 추출
elements = extract_pdf_elements(pdf_path, pdf_name)

# 페이지 단위로 Document 생성
source_path = os.path.join(pdf_path, pdf_name)
documents = convert_to_documents_by_page(elements, source_path)

# 예시 출력
with open("partition_pdf_output.txt", "w", encoding="utf-8") as f:
    f.write(documents[0].page_content)

### pdfplumber + pymupdf

In [ ]:
# pdfplumber로 PDF 파일을 열어서 표 추출 준비를 합니다.
with pdfplumber.open(file_path) as plumber_pdf:
    # PDF 각 페이지를 순회하며 인덱스와 페이지 객체를 가져옵니다.
    for i, page in enumerate(plumber_pdf.pages):
        page_num = i + 1  # 사람이 보는 페이지 번호 (0부터 시작하므로 +1)

        # (1) PyMuPDF를 이용해 동일한 페이지의 일반 텍스트(문단)를 추출합니다.
        # fitz_doc는 이미 외부에서 열려있는 PyMuPDF 문서 객체입니다.
        plain_text = fitz_doc[i].get_text("text").strip()
        # get_text("text")는 페이지 내 모든 텍스트를 줄바꿈 등 기본 포맷 유지하며 추출합니다.
        # strip()으로 앞뒤 공백 제거

        # (2) pdfplumber로 해당 페이지에서 표들을 추출합니다.
        tables = page.extract_tables()  # 표가 여러 개일 수 있어서 리스트 형태로 반환
        table_markdowns = []  # 추출된 표를 마크다운 형태로 변환해 담을 리스트

        # 각 표별로 반복하며 처리
        for idx, table in enumerate(tables):
            # 표가 비어있거나(빈 리스트) 헤더+데이터가 2행 미만인 경우 무시
            if not table or len(table) < 2:
                continue

            # 표 데이터를 pandas DataFrame으로 변환 (첫 행은 컬럼명으로 사용)
            df = pd.DataFrame(table[1:], columns=table[0])

            # DataFrame을 마크다운 표 형태로 변환 (문자열)
            markdown = df.to_markdown(index=False)

            # "[표 번호]"와 함께 마크다운 텍스트를 리스트에 저장
            table_markdowns.append(f"\n\n[표 {idx+1}]\n{markdown}")

        # (3) 문단 텍스트와 표 마크다운 텍스트를 하나로 합칩니다.
        combined_text = f"[페이지 {page_num}]\n\n{plain_text}"  # 기본 문단 텍스트

        # 표가 하나라도 있으면 각 표 마크다운을 본문 뒤에 붙임
        if table_markdowns:
            combined_text += "\n\n" + "\n\n".join(table_markdowns)
            # 여러 표가 있으면 각 표 사이에 줄바꿈 2번씩 추가해서 구분

        # (4) LangChain에서 쓸 Document 객체로 변환
        # page_content에 합쳐진 텍스트, metadata에 페이지 번호와 파일 경로 정보 포함
        doc = Document(
            page_content=combined_text,
            metadata={"page": page_num, "source": file_path}
        )

        # 완성된 Document를 문서 리스트에 추가
        documents.append(doc)


### PyMuPDF4LLMLoader

In [ ]:
import fitz
from langchain_pymupdf4llm import PyMuPDF4LLMLoader

file_path = './data/2025년 AI반도체 조기 상용화 및 AX실증 지원 사업 통합 공고문.pdf'

# strict=False 옵션을 주어 문서 열기 (에러 무시 가능)
fitz_doc = fitz.open(file_path, strict=False)

# PyMuPDF4LLMLoader에 이미 열린 문서를 직접 넘길 수 있다면 이렇게
loader = PyMuPDF4LLMLoader(fitz_doc)

documents = loader.load()

print(f"총 {len(documents)}개의 Document가 생성되었습니다.")
print("첫 문서 내용 일부:")
print(documents[0].page_content[:1000])
